Célula 1 – Montar Drive e verificar GPU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
print('CUDA disponível:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('Device:', torch.cuda.get_device_name(0))

Célula 2 – Clonar repositório e instalar pacote

In [ ]:
%cd /content
!git clone https://github.com/opontorno/DeepFeatureX-SN.git
%cd /content/DeepFeatureX-SN
!pip install -e .

Célula 3 – Extrair weights (.tar.gz)

In [ ]:
import os
import tarfile

# Caminho do .tar.gz no Drive
TAR_PATH = '/content/drive/MyDrive/tcc/dfx/weights.tar.gz'

# Pasta onde extrair (será o working_dir; deve conter "models" ao final)
WORKING_DIR = '/content/drive/MyDrive/tcc/dfx'
os.makedirs(WORKING_DIR, exist_ok=True)

with tarfile.open(TAR_PATH, 'r:gz') as tf:
    tf.extractall(WORKING_DIR)

# Conferir
!ls "{WORKING_DIR}"
!ls "{WORKING_DIR}/models/siamese-approach"
print('Working dir:', WORKING_DIR)

Célula 4 – Criar wd.py

In [ ]:
import os

REPO_ROOT = '/content/DeepFeatureX-SN'
# Pasta que contém a pasta "models" (não use .../dfx/weights)
WORKING_DIR = '/content/drive/MyDrive/tcc/dfx'

wd_content = f'''import os
working_dir = {repr(WORKING_DIR)}
'''
with open(os.path.join(REPO_ROOT, 'src', 'dfx', 'wd.py'), 'w') as f:
    f.write(wd_content)

print('wd.py criado. working_dir =', WORKING_DIR)

Célula 5 – Carregar modelo

In [ ]:
import sys
import os

REPO_ROOT = '/content/DeepFeatureX-SN'
sys.path.insert(0, os.path.join(REPO_ROOT, 'src'))

import torch
from dfx import get_complete_model, get_trans, get_path

# Backbone disponível nos seus weights: densenet121 ou densenet161
BACKBONE = 'densenet121'
models_dir = get_path('models')

model = get_complete_model(BACKBONE, models_dir=models_dir)
weight_path = f'{models_dir}/complete/{BACKBONE}.pt'
model.load_state_dict(torch.load(weight_path, map_location='cpu'))
model.eval()

trans = get_trans(model_name=BACKBONE)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print('Modelo carregado. Backbone:', BACKBONE, '| Device:', device)

Célula 7 – Inferência (upload no Colab)

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from pathlib import Path

datasets_path = '/content/drive/MyDrive/tcc/dfx/teste_inferencia'
import torch
from dfx import dataset_for_robustness, get_trans, make_balanced

trans = get_trans(model_name='densenet121')
dataset = dataset_for_robustness(dset_dir=datasets_path, transforms=trans)

GROUP_SIZE = 12
LABELS = ['DM', 'GAN', 'Real']
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

def tensor_to_img(t):
    x = t.cpu().numpy().transpose(1, 2, 0)
    x = x * std + mean
    x = np.clip(x, 0, 1)
    return x

n_total = len(dataset)
if n_total == 0:
    print('Dataset vazio. Verifique o caminho:', datasets_path)
else:
    model.eval()
    num_groups = (n_total + GROUP_SIZE - 1) // GROUP_SIZE

    for g in range(num_groups):
        start = g * GROUP_SIZE
        end = min(start + GROUP_SIZE, n_total)
        n_show = end - start

        cols = 4
        rows = max(1, (n_show + cols - 1) // cols)
        fig, axes = plt.subplots(rows, cols, figsize=(3 * cols, 3 * rows))
        if n_show == 1:
            axes = np.array([axes])
        axes = axes.flatten()

        for i in range(n_show):
            idx = start + i
            img_tensor, true_class, _ = dataset[idx]
            true_class = int(true_class.item())
            # retrieves the name of the model that generated the img
            arch_name = Path(dataset.files[idx]['file']).parent.name

            x = img_tensor.unsqueeze(0).to(device)
            with torch.no_grad():
                logits = model(x)
                pred = logits.argmax(dim=1).item()
            img_display = tensor_to_img(img_tensor)
            axes[i].imshow(img_display)
            axes[i].axis('off')
            color = 'green' if pred == true_class else 'red'
            axes[i].set_title(f'Esperado: {LABELS[true_class]} ({arch_name}) | Pred: {LABELS[pred]}', fontsize=10, color=color)

        for j in range(n_show, len(axes)):
            axes[j].axis('off')

        plt.suptitle(f'Grupo {g+1}/{num_groups} — imagens {start+1}–{end} de {n_total}', fontsize=12)
        plt.tight_layout()
        plt.show()

### testando no proprio dataset do autor :

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from pathlib import Path

datasets_path = '/content/drive/MyDrive/tcc/dfx/datasets'
import torch
from dfx import dataset_for_robustness, get_trans, make_balanced

trans = get_trans(model_name='densenet121')
dataset = dataset_for_robustness(dset_dir=datasets_path, transforms=trans)

GROUP_SIZE = 12
LABELS = ['DM', 'GAN', 'Real']
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

def tensor_to_img(t):
    x = t.cpu().numpy().transpose(1, 2, 0)
    x = x * std + mean
    x = np.clip(x, 0, 1)
    return x

n_total = len(dataset)
if n_total == 0:
    print('Dataset vazio. Verifique o caminho:', datasets_path)
else:
    model.eval()
    num_groups = (n_total + GROUP_SIZE - 1) // GROUP_SIZE

    for g in range(num_groups):
        start = g * GROUP_SIZE
        end = min(start + GROUP_SIZE, n_total)
        n_show = end - start

        cols = 4
        rows = max(1, (n_show + cols - 1) // cols)
        fig, axes = plt.subplots(rows, cols, figsize=(3 * cols, 3 * rows))
        if n_show == 1:
            axes = np.array([axes])
        axes = axes.flatten()

        for i in range(n_show):
            idx = start + i
            img_tensor, true_class, _ = dataset[idx]
            true_class = int(true_class.item())
            # retrieves the name of the model that generated the img
            arch_name = Path(dataset.files[idx]['file']).parent.name

            x = img_tensor.unsqueeze(0).to(device)
            with torch.no_grad():
                logits = model(x)
                pred = logits.argmax(dim=1).item()
            img_display = tensor_to_img(img_tensor)
            axes[i].imshow(img_display)
            axes[i].axis('off')
            color = 'green' if pred == true_class else 'red'
            axes[i].set_title(f'Esperado: {LABELS[true_class]} ({arch_name}) | Pred: {LABELS[pred]}', fontsize=10, color=color)

        for j in range(n_show, len(axes)):
            axes[j].axis('off')

        plt.suptitle(f'Grupo {g+1}/{num_groups} — imagens {start+1}–{end} de {n_total}', fontsize=12)
        plt.tight_layout()
        plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')